In [35]:
import pyodbc
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Data Exracting

In [36]:
server_name = 'localhost' 
database_name = 'Retail_Staging'
trusted_connection = 'yes' 

In [37]:
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};" 
    f"SERVER={server_name};"
    f"DATABASE={database_name};"
    f"Trusted_Connection={trusted_connection};"
)

In [38]:
connection = pyodbc.connect(connection_string)

In [57]:
sql_query = "SELECT * FROM sales_data"

In [58]:
data = pd.read_sql_query(sql_query, connection)

C:\Users\hp\AppData\Local\Temp\ipykernel_4396\2307129927.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(sql_query, connection)


# Data Exploring

In [59]:
data.head()

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
0,O966977,C00397,P0022,3,39.25,2025-06-07,Delivered,PayPal,Central,0
1,O696648,C00236,P0023,5,18.92,2025-06-07,DELAYED,credit card,North,0
2,O202644,C00492,P0011,1,29.68,2025-06-07,delivered,Bank Transfer,North,0.15
3,O501803,C00031,P0003,1,32.76,2025-06-07,Cancelled,Credit Card,Central,0.2
4,O322242,C00495,P0016,1,47.62,2025-06-07,DELAYED,Credit Card,West,0.2


In [60]:
data.tail()

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
2995,O868860,C00233,P0001,5,43.40,2025-06-07,Delivered,Bank Transfer,West,0.2
2996,O949709,C00246,P0029,4,34.04,2025-06-07,DELAYED,Bank Transfer,West,0.2
2997,O763639,C00182,P0026,1,42.34,2025-06-07,Delivered,credit card,South,0
2998,O753958,C00074,P0003,5,35.96,2025-06-07,Delivered,Credit Card,Central,0
2999,O929624,C00405,P0004,3,43.23,2025-06-07,delivered,Credit Card,West,0.1


In [61]:
data.describe()

,unit_price,order_date
count,2999.000000,2997
mean,28.945388,2025-06-07 00:00:00.000000256
min,8.000000,2025-06-07 00:00:00
25%,18.830000,2025-06-07 00:00:00
50%,28.630000,2025-06-07 00:00:00
75%,39.640000,2025-06-07 00:00:00
max,49.990000,2025-06-07 00:00:00
std,12.085893,NaN


In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          2999 non-null   object        
 1   customer_id       2998 non-null   object        
 2   product_id        2995 non-null   object        
 3   quantity          2997 non-null   object        
 4   unit_price        2999 non-null   float64       
 5   order_date        2997 non-null   datetime64[ns]
 6   delivery_status   2997 non-null   object        
 7   payment_method    2997 non-null   object        
 8   region            3000 non-null   object        
 9   discount_applied  2483 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 234.5+ KB


In [63]:
data.dtypes

order_id                    object
customer_id                 object
product_id                  object
quantity                    object
unit_price                 float64
order_date          datetime64[ns]
delivery_status             object
payment_method              object
region                      object
discount_applied            object
dtype: object

In [64]:
data.shape

(3000, 10)

# Data Cleaning

## Null Check

In [65]:
data.isnull().sum()

order_id              1
customer_id           2
product_id            5
quantity              3
unit_price            1
order_date            3
delivery_status       3
payment_method        3
region                0
discount_applied    517
dtype: int64

In [66]:
data[data['customer_id'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
851,O663482,None,P0029,3,18.89,2025-06-07,Cancelled,PayPal,West,0.05
1707,O735004,None,P0017,2,12.18,2025-06-07,delivered,Bank Transfer,Central,None


In [67]:
data[data['order_id'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
862,None,C00237,P0013,1,39.26,2025-06-07,Delayed,Credit Card,West,0.2


In [68]:
data[data['product_id'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
569,O711887,C00125,None,5,36.65,2025-06-07,Cancelled,Bank Transfer,South,0.15
613,O231318,C00251,None,5,47.51,2025-06-07,Delivered,PayPal,West,0.2
1267,O327646,C00451,None,4,29.16,2025-06-07,Delayed,credit card,West,0.15
1841,O218065,C00324,None,5,25.61,2025-06-07,Delayed,PayPal,North,0.2
2803,O140946,C00251,None,3,22.20,2025-06-07,Delivered,PayPal,East,0.05


In [69]:
data[data['quantity'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
1113,O718743,C00209,P0024,None,43.59,2025-06-07,Delivered,credit card,Central,0
1712,O490113,C00286,P0010,None,13.49,2025-06-07,DELAYED,Bank Transfer,South,0.2
2223,O189925,C00098,P0013,None,46.37,2025-06-07,Delivered,Bank Transfer,South,None


In [70]:
data[data['unit_price'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
496,O837867,C00350,P0028,4,NaN,2025-06-07,delivered,Credit Card,East,0.15


In [71]:
data[data['order_date'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
956,O715915,C00327,P0006,5,42.03,NaT,Cancelled,Bank Transfer,Central,0
1373,O746155,C00158,P0003,2,45.43,NaT,Delivered,credit card,Central,None
2090,O218231,C00324,P0014,4,12.76,NaT,Delayed,credit card,East,0.15


In [72]:
data[data['delivery_status'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
601,O112478,C00471,P0025,2,9.00,2025-06-07,None,Credit Card,North,0.2
733,O697006,C00324,P0015,5,42.49,2025-06-07,None,credit card,Central,None
1124,O239188,C00423,P0012,4,37.82,2025-06-07,None,credit card,East,0.05


In [73]:
data[data['payment_method'].isnull()]

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
858,O687548,C00500,P0004,2,45.04,2025-06-07,Cancelled,None,West,0.15
963,O912757,C00248,P0025,1,29.30,2025-06-07,DELAYED,None,East,0.05
1954,O331280,C00221,P0020,5,42.27,2025-06-07,DELAYED,None,West,0.15


In [75]:
data['discount_applied']= data['discount_applied'].fillna(0)

In [76]:
data= data.dropna()

In [77]:
data.isnull().sum()

order_id            0
customer_id         0
product_id          0
quantity            0
unit_price          0
order_date          0
delivery_status     0
payment_method      0
region              0
discount_applied    0
dtype: int64

In [78]:
data.shape

(2979, 10)

## Checking for Duplications

In [79]:
data.duplicated().sum()

np.int64(0)

## Check for Inconsistency

### quantity

In [80]:
data['quantity'].unique()

array(['3', '5', '1', '2', '4', 'three', 'five'], dtype=object)

In [81]:
data.loc[data['quantity'] == 'three', 'quantity'] = '3'
data.loc[data['quantity'] == 'five', 'quantity'] = '5'

### delivery_status

In [120]:
data['delivery_status'].unique()


array(['Delivered', 'Delayed', 'Cancelled'], dtype=object)

In [83]:
data.loc[data['delivery_status'] == 'delivered', 'delivery_status'] = 'Delivered'
data.loc[data['delivery_status'] == 'delrd', 'delivery_status'] = 'Delivered'

In [109]:
data.loc[data['delivery_status'] == ' DELAYED', 'delivery_status'] = 'Delayed'
data.loc[data['delivery_status'] == 'DELAYED', 'delivery_status'] = 'Delayed'
data.loc[data['delivery_status'] == 'delyd', 'delivery_status'] = 'Delayed'

In [119]:
data.loc[data['delivery_status'] == ' Cancelled ', 'delivery_status'] = 'Cancelled'
data.loc[data['delivery_status'] == 'Cancelled ', 'delivery_status'] = 'Cancelled'

### payment_method

In [86]:
data['payment_method'].unique()

array(['PayPal', 'credit card', 'Bank Transfer', 'Credit Card',
       'bank transfr'], dtype=object)

In [87]:
data.loc[data['payment_method'] == 'credit card', 'payment_method'] = 'Credit Card'

In [88]:
data.loc[data['payment_method'] == 'bank transfr', 'payment_method'] = 'Bank Transfer'

### region

In [89]:
data['region'].unique()

array(['Central', 'North', 'West', 'East', 'South', 'nrth'], dtype=object)

In [90]:
data.loc[data['region'] == 'nrth', 'region'] = 'North'

In [91]:
data['region'].value_counts(normalize=True) * 100

region
North      20.275260
Central    20.073850
East       20.040282
South      19.906009
West       19.704599
Name: proportion, dtype: float64

## Data Types Fixing

In [92]:
pd.to_datetime(data['order_date'])

0      2025-06-07
1      2025-06-07
2      2025-06-07
3      2025-06-07
4      2025-06-07
          ...    
2995   2025-06-07
2996   2025-06-07
2997   2025-06-07
2998   2025-06-07
2999   2025-06-07
Name: order_date, Length: 2979, dtype: datetime64[ns]

In [93]:
data['order_date']= pd.to_datetime(data['order_date'])

In [94]:
data['quantity'] = data['quantity'].astype(int)

In [95]:
data.head(10)

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
0,O966977,C00397,P0022,3,39.25,2025-06-07,Delivered,PayPal,Central,0
1,O696648,C00236,P0023,5,18.92,2025-06-07,DELAYED,Credit Card,North,0
2,O202644,C00492,P0011,1,29.68,2025-06-07,Delivered,Bank Transfer,North,0.15
3,O501803,C00031,P0003,1,32.76,2025-06-07,Cancelled,Credit Card,Central,0.2
4,O322242,C00495,P0016,1,47.62,2025-06-07,DELAYED,Credit Card,West,0.2
5,O190175,C00388,P0005,3,37.89,2025-06-07,Delayed,Bank Transfer,North,0.1
6,O272646,C00328,P0027,5,30.83,2025-06-07,Delivered,PayPal,Central,0.05
7,O411881,C00201,P0019,3,12.56,2025-06-07,Delivered,Credit Card,East,0
8,O170570,C00076,P0030,3,28.32,2025-06-07,Delivered,PayPal,Central,0.15
9,O619944,C00330,P0002,1,30.96,2025-06-07,DELAYED,Credit Card,South,0.15


# Loading The Data

In [96]:
dwh_conn = pyodbc.connect(
            "DRIVER={ODBC Driver 17 for SQL Server};"
            "SERVER=localhost;"
            "DATABASE=retail_dwh;"
            "Trusted_Connection=yes;"
        )

In [97]:
dwh_cursor = dwh_conn.cursor()

In [98]:
data.columns

Index(['order_id', 'customer_id', 'product_id', 'quantity', 'unit_price',
       'order_date', 'delivery_status', 'payment_method', 'region',
       'discount_applied'],
      dtype='object')

## Load into Payment Dimention

In [99]:
payment_methods = data['payment_method'].unique()

In [100]:
for method in payment_methods:
    dwh_cursor.execute("""
        INSERT INTO Dim_Payment (payment_method) 
        VALUES (?)
    """, method)

In [125]:
dwh_conn.commit()

## Load into Delivery Dimention

In [130]:
delivery_statuses = data['delivery_status'].unique()

In [131]:
delivery_statuses

array(['Delivered', 'Delayed', 'Cancelled'], dtype=object)

In [132]:
for status in delivery_statuses:
    dwh_cursor.execute("""
        INSERT INTO Dim_Delivery (delivery_status) 
        VALUES (?)
    """, status)

In [133]:
dwh_conn.commit()

## Load into Region Dimension

In [134]:
regions = data['region'].unique()

In [135]:
regions

array(['Central', 'North', 'West', 'East', 'South'], dtype=object)

In [136]:
for region in regions:
    dwh_cursor.execute("""
        INSERT INTO Dim_Region (region_name) 
        VALUES (?)
    """, region)

In [137]:
dwh_conn.commit()

## Load Data into Date Dimention

In [138]:
unique_dates = data['order_date'].dt.date.unique()

In [139]:
unique_dates

array([datetime.date(2025, 6, 7)], dtype=object)

In [140]:
for date_obj in unique_dates:
    date_id = date_obj.year * 10000 + date_obj.month * 100 + date_obj.day
    
    dwh_cursor.execute("""
        INSERT INTO Dim_Date (date_id, full_date, day, day_of_week, month, year)
        VALUES (?, ?, ?, ?, ?, ?)
    """,
    date_id,
    date_obj,
    date_obj.day,
    date_obj.strftime('%A'),  # Day name
    date_obj.month,
    date_obj.year)

In [141]:
dwh_conn.commit()

## Loading Data into The Fact Table

In [145]:
customer_in_sales = set(data['customer_id'].unique())

dwh_cursor.execute("SELECT customer_id FROM Dim_Customer")
existing_customers = set([row[0] for row in dwh_cursor.fetchall()])

missing_customers = customer_in_sales - existing_customers

if missing_customers:
    print(f"Found {len(missing_customers)} customers in sales but NOT in Dim_Customer")
    
    print("\nFiltering out missing customers")
    data = data[~data['customer_id'].isin(missing_customers)]
else:
    print("All customers in sales exist in Dim_Customer")

All customers in sales exist in Dim_Customer


In [146]:
data.shape

(2925, 10)

In [147]:
for index, row in data.iterrows():
    # Get date_id
    date_obj = row['order_date'].date()
    date_id = date_obj.year * 10000 + date_obj.month * 100 + date_obj.day
    
    # Get payment_method_id
    dwh_cursor.execute("""
        SELECT payment_method_id FROM Dim_Payment 
        WHERE payment_method = ?
    """, row['payment_method'])
    payment_result = dwh_cursor.fetchone()
    payment_method_id = payment_result[0]
    
    # Get delivery_status_id
    dwh_cursor.execute("""
        SELECT delivery_status_id FROM Dim_Delivery 
        WHERE delivery_status = ?
    """, row['delivery_status'])
    delivery_result = dwh_cursor.fetchone()
    delivery_status_id = delivery_result[0]
    
    # Get region_key
    dwh_cursor.execute("""
        SELECT region_key FROM Dim_Region 
        WHERE region_name = ?
    """, row['region'])
    region_result = dwh_cursor.fetchone()
    region_key = region_result[0]
    
    # Insert into fact_sales
    dwh_cursor.execute("""
        INSERT INTO fact_sales (
            order_id, product_id, customer_id, date_id,
            payment_method_id, region_key, delivery_status_id,
            quantity, unit_price, discount_applied
        )
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    row['order_id'],
    row['product_id'],
    row['customer_id'],
    date_id,
    payment_method_id,
    region_key,
    delivery_status_id,
    row['quantity'],
    row['unit_price'],
    row['discount_applied'])

In [148]:
dwh_conn.commit()

In [149]:
dwh_cursor.close()
dwh_conn.close()

# Saving File

In [150]:
data.head(10)

,order_id,customer_id,product_id,quantity,unit_price,order_date,delivery_status,payment_method,region,discount_applied
0,O966977,C00397,P0022,3,39.25,2025-06-07,Delivered,PayPal,Central,0
1,O696648,C00236,P0023,5,18.92,2025-06-07,Delayed,Credit Card,North,0
2,O202644,C00492,P0011,1,29.68,2025-06-07,Delivered,Bank Transfer,North,0.15
3,O501803,C00031,P0003,1,32.76,2025-06-07,Cancelled,Credit Card,Central,0.2
4,O322242,C00495,P0016,1,47.62,2025-06-07,Delayed,Credit Card,West,0.2
5,O190175,C00388,P0005,3,37.89,2025-06-07,Delayed,Bank Transfer,North,0.1
6,O272646,C00328,P0027,5,30.83,2025-06-07,Delivered,PayPal,Central,0.05
7,O411881,C00201,P0019,3,12.56,2025-06-07,Delivered,Credit Card,East,0
8,O170570,C00076,P0030,3,28.32,2025-06-07,Delivered,PayPal,Central,0.15
9,O619944,C00330,P0002,1,30.96,2025-06-07,Delayed,Credit Card,South,0.15


In [152]:
data.to_csv('Sales.csv')